In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import f_oneway
from scipy.stats import ttest_ind

import sys
sys.path.append("../scripts")

import preprocessing as dp
import data_extraction as de
import exploratory_analysis as eda

In [2]:
df = de.load_data('../data/csv.csv')

d:\Personal Files\KIAM Notebooks\KIAM-Week3\notebooks\../scripts\data_extraction.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


Data Loaded Successfully


In [3]:
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,3/1/2015 0:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0
1,145249,12827,5/1/2015 0:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0
2,145249,12827,7/1/2015 0:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0
3,145255,12827,5/1/2015 0:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0
4,145255,12827,7/1/2015 0:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               104888 non-null   object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

In [5]:
# duplicated values
df.duplicated().sum()

np.int64(0)

In [6]:
# Drop Duplicates
df.drop_duplicates(inplace=True)

#### Hypothesis Testing 

1. Null Hypothesis: There are no risk differences across provinces 

In [7]:
## Calculate claims to Premium ratio 

df['claims_to_premiums_ratio'] = df['TotalClaims'] / (df['TotalPremium'] + 1)

In [8]:
### Let's group the data into provinces

anova = [df[df['Province'] == province]['claims_to_premiums_ratio'] for province in df['Province'].unique()]


In [9]:
## Perform anova test

anova_result = f_oneway(*anova)

print('ANOVA Results for Provinces:', anova_result)

if anova_result.pvalue < 0.05:
    print("Reject the null hypothesis: There are significant risk differences across provinces.")
else:
    print("Fail to reject the null hypothesis: No significant risk differences across provinces.")

ANOVA Results for Provinces: F_onewayResult(statistic=np.float64(2.161069458248007), pvalue=np.float64(0.027241688082460597))
Reject the null hypothesis: There are significant risk differences across provinces.


2. There are no risk differences between zip codes

In [10]:
## Let's group the data randomly into 2 groups using the postal code
unique_postal_codes = df['PostalCode'].unique()

# Shuffle the postal codes
np.random.shuffle(unique_postal_codes)

# Split into two groups
split_index = len(unique_postal_codes) // 2
group1 = unique_postal_codes[:split_index]
group2 = unique_postal_codes[split_index:]

# Create a new column to indicate the group for each postal code
df['group'] = df['PostalCode'].apply(lambda x: 'Group 1' if x in group1 else 'Group 2')

# Check the grouping
print(df[['PostalCode', 'group']].sample(5))

        PostalCode    group
606821        8000  Group 2
918817         122  Group 2
560124        2410  Group 1
132488        1062  Group 1
357309        1057  Group 1


In [11]:
# Extract claims-to-premiums ratio for each group
group1_data = df[df['group'] == 'Group 1']['claims_to_premiums_ratio']
group2_data = df[df['group'] == 'Group 2']['claims_to_premiums_ratio']

# Perform independent t-test
t_test_result = ttest_ind(group1_data, group2_data)
print('T-test Results for Gender Differences:', t_test_result)

if t_test_result.pvalue < 0.05:
    print("Reject the null hypothesis: There are significant risk differences between zip codes.")
else:
    print("Fail to reject the null hypothesis: No significant risk differences between zip codes.")

T-test Results for Gender Differences: TtestResult(statistic=np.float64(-1.1504962683296014), pvalue=np.float64(0.2499398053870172), df=np.float64(1000096.0))
Fail to reject the null hypothesis: No significant risk differences between zip codes.


3. There are no significant margin (profit) difference between zip codes 

In [12]:
## Let's assume profit is calculated as the difference between Premium and claims

df['Profit'] = df['TotalPremium'] - df['TotalClaims']

In [13]:
# Extract profit for each group
group1_data = df[df['group'] == 'Group 1']['Profit']
group2_data = df[df['group'] == 'Group 2']['Profit']

# Perform independent t-test
t_test_result = ttest_ind(group1_data, group2_data)
print('T-test Results for Gender Differences:', t_test_result)

if t_test_result.pvalue < 0.05:
    print("Reject the null hypothesis: There are significant margin (profit) difference between zip codes")
else:
    print("Fail to reject the null hypothesis: There are no significant margin (profit) difference between zip codes.")

T-test Results for Gender Differences: TtestResult(statistic=np.float64(0.32526574016986604), pvalue=np.float64(0.7449800257219982), df=np.float64(1000096.0))
Fail to reject the null hypothesis: There are no significant margin (profit) difference between zip codes.


4. There are not significant risk difference between Women and Men

In [14]:
# Separate data by gender
risk_men = df[df['Gender'] == 'Male']['claims_to_premiums_ratio']
risk_women = df[df['Gender'] == 'Female']['claims_to_premiums_ratio']

# Perform independent t-test
t_test_result = ttest_ind(risk_men, risk_women)
print('T-test Results for Gender Differences:', t_test_result)

if t_test_result.pvalue < 0.05:
    print("Reject the null hypothesis: There are significant risk differences between women and men.")
else:
    print("Fail to reject the null hypothesis: No significant risk differences between women and men.")

T-test Results for Gender Differences: TtestResult(statistic=np.float64(0.6959441469250746), pvalue=np.float64(0.4864670714626016), df=np.float64(49570.0))
Fail to reject the null hypothesis: No significant risk differences between women and men.
